<a href="https://colab.research.google.com/github/EvagAIML/AIML/blob/main/Fullcode_ML_additional_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem Statement

### Business Context

Airbnb is an online platform that allows people to rent short-term accommodation. This ranges from regular people with a spare bedroom to property management firms who lease multiple rentals. On the one side, Airbnb enables owners to list their space and earn rental money. On the other side, it provides travelers easy access to renting private homes.

Airbnb receives commissions from two sources upon every booking, namely from the hosts and guests. For every booking, Airbnb charges the guest 6-12% of the booking fee. Moreover, Airbnb charges the host 3% for every successful transaction.

As a senior data scientist at Airbnb, you have to come up with a pricing model that can effectively predict the Rent for accommodation and can help hosts, travelers, and also the business in devising profitable strategies.

### Objective

To explore and visualize the data, build a linear regression model to predict the prices of Airbnb rental rooms, and generate a set of insights and recommendations that will help the business.

### Data Description

The data contains information about the different types of rental rooms offered by Airbnb over a fixed period of time. The detailed data dictionary is given below.

**Data Dictionary**

- id: Property ID
- room_type: Type of Room in the property
- accommodates: How many adults can this property accommodate
- bathrooms: Number of bathrooms on the property
- cancellation_policy: Cancellation policy of the property
- cleaning_fee: This denotes whether the property cleaning fee is included in the rent or not
- instant_bookable: It indicates whether an instant booking facility is available or not
- review_scores_rating: Review rating score of the property
- bedrooms: Number of bedrooms in the property
- beds: Total number of beds in the property
- log_price: Log of the rental price of the property for a fixed period. [If the price is 12000 dollars, then log_price represents log(12000)]

## Importing necessary libraries

In [1]:
# ✅ Install only compatible modern packages in Colab
!pip install -q --upgrade \
    numpy==1.26.4 \
    pandas==2.2.2 \
    matplotlib==3.8.4 \
    seaborn==0.13.2 \
    scikit-learn==1.5.0


:**Note**: *After running the above cell, kindly restart the notebook kernel and run all cells sequentially from the start again.*

In [2]:
# Libraries to help with reading and manipulating data
import numpy as np
import pandas as pd

# Libraries to help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

# Removes the limit for the number of displayed columns
pd.set_option("display.max_columns", None)
# Sets the limit for the number of displayed rows
pd.set_option("display.max_rows", 200)

# to split the data into train and test
from sklearn.model_selection import train_test_split

# to build linear regression_model
from sklearn.linear_model import LinearRegression

# to check model performance
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# to suppress warnings
import warnings

warnings.filterwarnings("ignore")

## Loading the dataset

In [3]:
# GitHub raw link
url = "https://raw.githubusercontent.com/EvagAIML/AIML/main/Airbnb.csv"

# Load the dataset
df = pd.read_csv(url)

# Preview the dataset
df.head()

,id,room_type,accommodates,bathrooms,cancellation_policy,cleaning_fee,instant_bookable,review_scores_rating,bedrooms,beds,log_price
0,6901257,Entire home/apt,3.0,1.0,strict,True,f,100.0,1.0,1.0,5.010635
1,6304928,Entire home/apt,7.0,1.0,strict,True,t,93.0,3.0,3.0,5.129899
2,7919400,Entire home/apt,5.0,1.0,moderate,True,t,92.0,1.0,3.0,4.976734
3,13418779,Entire home/apt,4.0,1.0,flexible,True,f,NaN,2.0,2.0,6.620073
4,3808709,Entire home/apt,2.0,1.0,moderate,True,t,40.0,0.0,1.0,4.744932


## Data Overview

- Observations
- Sanity checks

In [6]:
# Shape of the dataset
print("Shape of the dataset:", df.shape)

# Data type and missing value summary
print("\n--- Data Info ---")
df.info()

# First 5 rows of data
print("\n--- Sample Records ---")
display(df.head())

# Summary statistics for all columns
print("\n--- Statistical Summary ---")
display(df.describe(include='all'))

# Checking missing values
print("\n--- Missing Value Count ---")
print(df.isnull().sum())

# Checking for duplicates
print("\n--- Duplicate Rows ---")
print(df.duplicated().sum())

# Checking for unrealistic zero or negative values
cols_to_check = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'log_price']
print("\n--- Records with Zero or Negative Key Values ---")
for col in cols_to_check:
    if col in df.columns:
        print(f"\n{col}:")
        display(df[df[col] <= 0])

Shape of the dataset: (74111, 11)

--- Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74111 entries, 0 to 74110
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    74111 non-null  int64  
 1   room_type             74106 non-null  object 
 2   accommodates          74108 non-null  float64
 3   bathrooms             73908 non-null  float64
 4   cancellation_policy   74103 non-null  object 
 5   cleaning_fee          74107 non-null  object 
 6   instant_bookable      74111 non-null  object 
 7   review_scores_rating  57389 non-null  float64
 8   bedrooms              74019 non-null  float64
 9   beds                  73980 non-null  float64
 10  log_price             74111 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 6.2+ MB

--- Sample Records ---


,id,room_type,accommodates,bathrooms,cancellation_policy,cleaning_fee,instant_bookable,review_scores_rating,bedrooms,beds,log_price
0,6901257,Entire home/apt,3.0,1.0,strict,True,f,100.0,1.0,1.0,5.010635
1,6304928,Entire home/apt,7.0,1.0,strict,True,t,93.0,3.0,3.0,5.129899
2,7919400,Entire home/apt,5.0,1.0,moderate,True,t,92.0,1.0,3.0,4.976734
3,13418779,Entire home/apt,4.0,1.0,flexible,True,f,NaN,2.0,2.0,6.620073
4,3808709,Entire home/apt,2.0,1.0,moderate,True,t,40.0,0.0,1.0,4.744932



--- Statistical Summary ---


,id,room_type,accommodates,bathrooms,cancellation_policy,cleaning_fee,instant_bookable,review_scores_rating,bedrooms,beds,log_price
count,7.411100e+04,74106,74108.000000,73908.000000,74103,74107,74111,57389.000000,74019.000000,73980.000000,74111.000000
unique,NaN,3,NaN,NaN,3,2,2,NaN,NaN,NaN,NaN
top,NaN,Entire home/apt,NaN,NaN,strict,True,f,NaN,NaN,NaN,NaN
freq,NaN,41308,NaN,NaN,32500,54399,54660,NaN,NaN,NaN,NaN
mean,1.126662e+07,NaN,3.155125,1.235272,NaN,NaN,NaN,94.067365,1.265797,1.710868,4.782069
std,6.081735e+06,NaN,2.153603,0.582054,NaN,NaN,NaN,7.836556,0.852149,1.254142,0.717394
min,3.440000e+02,NaN,1.000000,0.000000,NaN,NaN,NaN,20.000000,0.000000,0.000000,0.000000
25%,6.261964e+06,NaN,2.000000,1.000000,NaN,NaN,NaN,92.000000,1.000000,1.000000,4.317488
50%,1.225415e+07,NaN,2.000000,1.000000,NaN,NaN,NaN,96.000000,1.000000,1.000000,4.709530
75%,1.640226e+07,NaN,4.000000,1.000000,NaN,NaN,NaN,100.000000,1.000000,2.000000,5.220356



--- Missing Value Count ---
id                          0
room_type                   5
accommodates                3
bathrooms                 203
cancellation_policy         8
cleaning_fee                4
instant_bookable            0
review_scores_rating    16722
bedrooms                   92
beds                      131
log_price                   0
dtype: int64

--- Duplicate Rows ---
0

--- Records with Zero or Negative Key Values ---

accommodates:


...

,id,room_type,accommodates,bathrooms,cancellation_policy,cleaning_fee,instant_bookable,review_scores_rating,bedrooms,beds,log_price
4,3808709,Entire home/apt,2.0,1.0,moderate,True,t,40.0,0.0,1.0,4.744932
55,6443678,Entire home/apt,5.0,1.0,strict,True,f,100.0,0.0,2.0,4.744932
64,5806687,Entire home/apt,1.0,1.0,flexible,False,f,NaN,0.0,1.0,4.700480
69,11095719,Entire home/apt,2.0,1.0,flexible,True,f,100.0,0.0,1.0,4.499810
96,4576821,Entire home/apt,1.0,1.0,flexible,True,f,100.0,0.0,1.0,4.248495
...,...,...,...,...,...,...,...,...,...,...,...
74086,224510,Private room,2.0,1.0,strict,True,f,95.0,0.0,1.0,4.174387
74087,15042259,Entire home/apt,4.0,1.0,strict,True,t,100.0,0.0,2.0,4.477337
74091,3779249,Entire home/apt,2.0,1.0,moderate,False,f,100.0,0.0,1.0,5.010635
74098,8342838,Entire home/apt,4.0,1.0,flexible,True,f,88.0,0.0,2.0,4.553877



beds:


,id,room_type,accommodates,bathrooms,cancellation_policy,cleaning_fee,instant_bookable,review_scores_rating,bedrooms,beds,log_price
13672,7922378,Entire home/apt,2.0,1.0,strict,True,f,NaN,1.0,0.0,5.129899
24062,20269147,Entire home/apt,2.0,1.0,strict,False,t,NaN,0.0,0.0,5.783825
39057,8684344,Entire home/apt,2.0,1.0,strict,True,t,94.0,0.0,0.0,4.691348
55285,15419976,Entire home/apt,4.0,1.0,strict,False,t,NaN,1.0,0.0,5.991465



log_price:


,id,room_type,accommodates,bathrooms,cancellation_policy,cleaning_fee,instant_bookable,review_scores_rating,bedrooms,beds,log_price
11632,17972519,Shared room,1.0,1.0,strict,True,t,87.0,1.0,1.0,0.0


## Exploratory Data Analysis (EDA)

:- EDA is an important part of any project involving data.
- It is important to investigate and understand the data better before building a model with it.
- A few questions have been mentioned below which will help you approach the analysis in the right manner and generate insights from the data.
- A thorough analysis of the data, in addition to the questions mentioned below, should be done.

**Questions**:

1. How are the prices (log_price) of properties distributed?
2. What is the market share of different cancellation policies?
3. How does the price (log_price) of a property vary by the number of bedrooms?
4. How does the price (log_price) of a property vary by the number of accommodates?
5. What are the attributes that have a strong correlation with the price (log_price) of the property?



## Data Preprocessing

- Missing value treatment
- Feature engineering (if needed)
- Outlier detection and treatment (if needed)
- Preparing data for modeling
- Any other preprocessing steps (if needed)

## Model Building - Linear Regression

## Model Performance Check

---



## Actionable Insights and Recommendations

-


___